In [ ]:
# default_exp db

In [ ]:
%reload_ext autoreload
%autoreload 2

# 数据库
> Redis，MongoDB如何选择

为什么要用数据库？因为保存为单独文件的读写性能非常的弱，而且数据要全部加载到内存中才能进行分析，而数据库则具有更好的性能

## Redis [官网](https://redis.io/)
>  [线上体验redis](http://try.redis.io/)

Redis优势（可以用作数据库、缓存和消息中间件）：
* 速度快：性能极高 – Redis能读的速度是110000次/s,写的速度是81000次/s 
* 持久化：可以通过RDB和AOF两种方式，定期/实时将数据持久化到磁盘上
* 支持多种数据结构：支持五种基本的数据结构，分别是String（字符串），Hash（哈希），List（列表），Set（集合），Zset（即Sorted Set有序集合）
* 集群&主从复制

安装步骤：
* mac上安装redis： `brew install redis`
* 安装python中调用redis的插件：`pip install redis` [redis-py文档](https://redis-py.readthedocs.io/en/latest/)   |  [安装教程](https://redislabs.com/lp/python-redis/)
* 命令行中启动redis服务：`redis-server`


In [ ]:
# export
import redis 

In [ ]:
# export
import requests,json,re,random,sys
from bs4 import BeautifulSoup,Tag,NavigableString

from crawler_from_scratch.proxy import *

from concurrent.futures import ThreadPoolExecutor
import pandas as pd


In [ ]:
r = redis.Redis(host='localhost', port=6379, decode_responses=True)   # host是redis主机，需要redis服务端和客户端都启动 redis默认端口是6379
r.set('name', 'neo')
r.get('name')

### 连接池(connection pool) [原理](https://www.u3v3.com/ar/1346)
* 通常情况：连接-释放，完成一次数据读写
* 高并发：频繁的连接创建和释放对性能会有较高的影响
* 连接池：创建的连接不立即释放，而更多的请求发生时，会额外创建新的连接，这样多个就不会因为连接和释放影响数据读写性能

In [ ]:
pool = redis.ConnectionPool(host='localhost', port=6379, decode_responses=True)
rdb = redis.Redis(connection_pool=pool)

In [ ]:
rdb.hmset("001", {"name": "neo", "sex": "male"})
rdb.hgetall('001')

### 数据存储
redis会定期将内存中的数据同步到磁盘中，默认为启动目录下的`dump.rdb`;
如果要修改存储方式和文件，需要修改`redis.conf`文件

## 爬下厨房菜谱
> 因为平时做菜，想找到哪些材料，会明显提升菜的受欢迎程度，所以打算先爬所有热门菜谱的id，然后再爬详情页的配料表和互动数据

### 先从每月50最佳菜谱开始

In [ ]:
ip_pool = get_healthy_ip_pool('http')
ip_pool

In [ ]:
smart_proxy_get('http://www.xiachufang.com/explore/monthhonor/202002/',ip_pool)

http://84.17.47.190:80 健康值更新为： 26
http://65.52.158.214:80 健康值更新为： 54
成功: http://www.xiachufang.com/explore/monthhonor/202002/


<Response [200]>

### 获取待访问的url列表

In [ ]:
def get_monthhonor_urls():
    url_list = []
    for year in range(2011,2021):
        for month in range(1,13):
            if month < 10:
                month = f'0{month}'
            url = f'http://www.xiachufang.com/explore/monthhonor/{year}{month}/'
            url_list.append(url)


    url_page_list = []
    for url in url_list[2:-10]:
        for page in [1,2]:
            url_page_list.append(f'{url}?page={page}')

    print('url_counts:',len(url_page_list))
    return url_page_list

In [ ]:
def crawl_recipe_ids(url):
    res = smart_proxy_get(url,ip_pool)
    soup = BeautifulSoup(res.text,'lxml')
    a_list = soup.find_all(lambda tag : tag.name == 'a' and tag.parent.name == 'p' and 'name' in tag.parent['class'])
    id_list = [a['href'].split('/')[2] for a in a_list]
    id_dict = dict(zip(id_list,['' for i in range(len(id_list))]))
    
    rdb = redis.Redis(host='localhost', port=6379, decode_responses=True)   # host是redis主机，需要redis服务端和客户端都启动 redis默认端口是6379
    rdb.hmset('ids',id_dict)
    
    print(len(id_list),'个id保存成功', url)

In [ ]:
url = 'http://www.xiachufang.com/explore/monthhonor/202002/'
crawl_recipe_ids(url)

http://163.172.152.52:8811 健康值更新为： 55
成功: http://www.xiachufang.com/explore/monthhonor/202002/


In [ ]:
def crawl_monthhonor_recipe_ids():
    urls = get_monthhonor_urls()
    with ThreadPoolExecutor(max_workers=20) as executor:
        executor.map(crawl_recipe_ids,urls)
    rdb = redis.Redis(host='localhost', port=6379, decode_responses=True)   # host是redis主机，需要redis服务端和客户端都启动 redis默认端口是6379
    counts = rdb.hlen('ids')    
    print('现存id数：',counts)

In [ ]:
crawl_monthhonor_recipe_ids()

url_counts: 216
http://180.97.33.218:80 健康值更新为： 26
http://180.97.33.218:80 健康值更新为： 13
http://111.206.37.68:80 健康值更新为： 26
http://119.41.236.180:8010 健康值更新为： 54
成功: http://www.xiachufang.com/explore/monthhonor/201105/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201105/?page=2
http://119.41.236.180:8010 健康值更新为： 55
成功: http://www.xiachufang.com/explore/monthhonor/201111/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201111/?page=2
http://119.41.236.180:8010 健康值更新为： 56
成功: http://www.xiachufang.com/explore/monthhonor/201109/?page=1
http://119.41.236.180:8010 健康值更新为： 57
成功: http://www.xiachufang.com/explore/monthhonor/201201/?page=1
http://118.69.50.154:80 健康值更新为： 54
成功: http://www.xiachufang.com/explore/monthhonor/201107/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201109/?page=1
http://65.52.158.214:8025http://65.52.158.214:80 健康值更新为： 56
成功: http://www.xiachufang.com/explore/monthhonor/201107/?page=1
 个id保存成功 http://www.xiachufang.com/

25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201307/?page=1
http://119.41.236.180:8010 健康值更新为： 64
成功: http://www.xiachufang.com/explore/monthhonor/201309/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201106/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201304/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201306/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201309/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201112/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201307/?page=2
http://185.186.77.197:80 健康值更新为： 55
成功: http://www.xiachufang.com/explore/monthhonor/201308/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201308/?page=2
http://84.17.47.192:80 健康值更新为： 62
成功: http://www.xiachufang.com/explore/monthhonor/201309/?page=2
http://118.69.50.154:80 健康值更新为： 56
成功: http://www.xiachufang.com/explore/monthhonor/201105/?page=1
http://84.17.47.192:80 健康值更新为： 63

25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201505/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201503/?page=2
http://61.135.185.152:80 健康值更新为： 26
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201501/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201504/?page=2
http://61.135.185.152:80 健康值更新为： 13
http://185.186.77.197:80 健康值更新为： 57
成功: http://www.xiachufang.com/explore/monthhonor/201304/?page=1
http://119.41.236.180:8010 健康值更新为： 68
成功: http://www.xiachufang.com/explore/monthhonor/201502/?page=1
http://163.172.152.52:8811http://111.206.37.244:80  健康值更新为： 26
健康值更新为： 65
成功: http://www.xiachufang.com/explore/monthhonor/201507/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201212/?page=1
http://159.203.166.41:8080 健康值更新为： 13
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201506/?page=2
http://39.137.95.71:80 健康值更新为： 60
成功: http://www.xiachufang.com/explore/monthhonor/201512/?page=2
http://84.17.47.192:80 健康值

25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201703/?page=1
http://84.17.47.192:80http://39.137.95.71:80 健康值更新为： 71
成功: http://www.xiachufang.com/explore/monthhonor/201701/?page=1
http://84.17.47.192:80 健康值更新为： 72
成功: http://www.xiachufang.com/explore/monthhonor/201612/?page=1
 http://119.41.236.180:8010健康值更新为： 68
成功: http://www.xiachufang.com/explore/monthhonor/201711/?page=1
 健康值更新为： 73
成功: http://www.xiachufang.com/explore/monthhonor/201708/?page=2
http://65.52.158.214:80 健康值更新为： 77
成功: http://www.xiachufang.com/explore/monthhonor/201706/?page=2
http://185.186.77.197:80 健康值更新为： 63
成功: http://www.xiachufang.com/explore/monthhonor/201705/?page=1
http://65.52.158.214:80 健康值更新为： 78
成功: http://www.xiachufang.com/explore/monthhonor/201705/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201702/?page=2
http://84.17.47.185:80 健康值更新为： 66
成功: http://www.xiachufang.com/explore/monthhonor/201707/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201609/?pa

25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201806/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201809/?page=1
http://84.17.47.204:80 健康值更新为： 79
成功: http://www.xiachufang.com/explore/monthhonor/201903/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201901/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201902/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201905/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201902/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201904/?page=2
http://65.52.158.214:80 健康值更新为： 81
成功: http://www.xiachufang.com/explore/monthhonor/201906/?page=1
http://84.17.47.192:80 健康值更新为： 78
成功: http://www.xiachufang.com/explore/monthhonor/201905/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201907/?page=2
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201903/?page=1
25 个id保存成功 http://www.xiachufang.com/explore/monthhonor/201904/?pa

### 抓取菜谱详情页信息
>菜谱名
>创建者
>创建时间
>用料
>受欢迎度
>评分
>html

In [ ]:
res = smart_proxy_get('http://www.xiachufang.com/recipe/101770952/',ip_pool)
soup = BeautifulSoup(res.text,'lxml')


http://119.41.236.180:8010 健康值更新为： 78
成功: http://www.xiachufang.com/recipe/101770952/


In [ ]:
rdb.hset('101770952','html',res.text)

0

In [ ]:
def _crawl_recipe_content(_id):
    url = 'http://www.xiachufang.com/recipe/'+_id
    res = smart_proxy_get(url,ip_pool)
    if res.status_code == 200:
        rdb = redis.Redis(host='localhost', port=6379, decode_responses=True)   # host是redis主机，需要redis服务端和客户端都启动 redis默认端口是6379
        rdb.hset('html',_id,res.text) 
        print(_id,'进度：',format(rdb.hlen('html')/rdb.hlen('ids'),'.2%'))
    
def crawl_monthhonor_recipe_content():
    rdb = redis.Redis(host='localhost', port=6379, decode_responses=True)   # host是redis主机，需要redis服务端和客户端都启动 redis默认端口是6379
    ids = rdb.hkeys('ids')   
    with ThreadPoolExecutor(max_workers=50) as executor:
        executor.map(_crawl_recipe_content,ids)

In [ ]:
crawl_monthhonor_recipe_content()

http://39.137.95.71:80 健康值更新为： 411
成功: http://www.xiachufang.com/recipe/104228518
104228518 进度： 0.64%
http://39.137.95.70:80 健康值更新为： 326
成功: http://www.xiachufang.com/recipe/17851
17851 进度： 0.64%
http://39.137.95.71:80 健康值更新为： 412
成功: http://www.xiachufang.com/recipe/1016616
1016616 进度： 0.64%
http://39.137.95.70:80 健康值更新为： 327
成功: http://www.xiachufang.com/recipe/100534178
100534178 进度： 0.64%
http://39.137.95.71:80 健康值更新为： 413
成功: http://www.xiachufang.com/recipe/100422462
100422462 进度： 0.64%
http://39.137.95.70:80 健康值更新为： 328
成功: http://www.xiachufang.com/recipe/101865764
101865764 进度： 0.64%
http://39.137.95.70:80 健康值更新为： 329
成功: http://www.xiachufang.com/recipe/103345131
103345131 进度： 0.67%
http://39.137.95.71:80 健康值更新为： 414
成功: http://www.xiachufang.com/recipe/102924920
102924920 进度： 0.70%
http://39.137.95.70:80 健康值更新为： 330
成功: http://www.xiachufang.com/recipe/100415378
100415378 进度： 0.73%
http://39.137.95.70:80 健康值更新为： 331
成功: http://www.xiachufang.com/recipe/99804
99804 进度： 0.76%


http://163.172.136.226:8811 健康值更新为： 238
成功: http://www.xiachufang.com/recipe/101776405
101776405 进度： 2.54%
http://65.52.158.214:80 健康值更新为： 341
成功: http://www.xiachufang.com/recipe/100132107
100132107 进度： 2.57%
http://84.17.47.204:80 健康值更新为： 328
成功: http://www.xiachufang.com/recipe/1003015
1003015 进度： 2.57%
http://51.158.165.18:8811 健康值更新为： 229
成功: http://www.xiachufang.com/recipe/100619525
100619525 进度： 2.61%
http://84.17.47.185:80 健康值更新为： 232
成功: http://www.xiachufang.com/recipe/231924
231924 进度： 2.64%
http://51.158.165.18:8811 健康值更新为： 230
成功: http://www.xiachufang.com/recipe/1078283
1078283 进度： 2.67%
http://163.172.152.52:8811 健康值更新为： 422
成功: http://www.xiachufang.com/recipe/257903
257903 进度： 2.70%
http://163.172.152.52:8811 健康值更新为： 423
成功: http://www.xiachufang.com/recipe/101853958
101853958 进度： 2.73%
http://163.172.136.226:8811 健康值更新为： 239
成功: http://www.xiachufang.com/recipe/1008253
1008253 进度： 2.76%
http://118.69.50.154:80 健康值更新为： 289
成功: http://www.xiachufang.com/recipe/10222979

http://84.17.47.204:80 健康值更新为： 338
成功: http://www.xiachufang.com/recipe/100021799
100021799 进度： 4.97%
http://84.17.47.204:80 健康值更新为： 339
成功: http://www.xiachufang.com/recipe/103397338
103397338 进度： 5.00%
http://39.137.95.70:80 健康值更新为： 347
成功: http://www.xiachufang.com/recipe/87226
87226 进度： 5.03%
http://163.172.136.226:8811 健康值更新为： 247
成功: http://www.xiachufang.com/recipe/101806666
101806666 进度： 5.06%
http://39.137.95.71:80 健康值更新为： 427
成功: http://www.xiachufang.com/recipe/100464836
100464836 进度： 5.09%
http://163.172.136.226:8811 健康值更新为： 248
成功: http://www.xiachufang.com/recipe/103713450
103713450 进度： 5.12%
http://65.52.158.214:80 健康值更新为： 351
成功: http://www.xiachufang.com/recipe/1069116
1069116 进度： 5.15%
http://84.17.47.204:80 健康值更新为： 340
成功: http://www.xiachufang.com/recipe/100030417
100030417 进度： 5.18%
http://118.69.50.154:80 健康值更新为： 297
成功: http://www.xiachufang.com/recipe/100136747
100136747 进度： 5.21%
http://185.186.77.197:80 健康值更新为： 104
http://39.137.95.71:80 健康值更新为： 428
成功: http:/

101843648 进度： 7.51%
101892335 进度： 7.54%
http://51.158.165.18:8811 健康值更新为： 246
成功: http://www.xiachufang.com/recipe/100105035
100105035 进度： 7.57%
http://84.17.47.185:80 健康值更新为： 137
成功: http://www.xiachufang.com/recipe/1012137
1012137 进度： 7.60%
http://39.137.95.70:80 健康值更新为： 352
成功: http://www.xiachufang.com/recipe/102177434
102177434 进度： 7.63%
http://51.158.165.18:8811 http://84.17.47.204:80 健康值更新为： 348
成功: http://www.xiachufang.com/recipe/1039740
健康值更新为： 247
成功: http://www.xiachufang.com/recipe/103796281
1039740 进度： 7.69%
103796281 进度： 7.69%
http://84.17.47.185:80 健康值更新为： 138
成功: http://www.xiachufang.com/recipe/100513125
http://65.52.158.214:80 健康值更新为： 358
成功: http://www.xiachufang.com/recipe/268066
100513125 进度： 7.72%
268066 进度： 7.76%
http://39.137.95.71:80 健康值更新为： 436
成功: http://www.xiachufang.com/recipe/17953
17953 进度： 7.79%
http://185.186.77.197:80 健康值更新为： 111
成功: http://www.xiachufang.com/recipe/100378700
100378700 进度： 7.82%
http://51.158.165.18:8811 健康值更新为： 248
成功: http://www.xi

267078 进度： 10.03%
http://185.186.77.197:80 健康值更新为： 125
成功: http://www.xiachufang.com/recipe/100101941
100101941 进度： 10.06%
http://65.52.158.214:80 健康值更新为： 370
成功: http://www.xiachufang.com/recipe/102835644
102835644 进度： 10.09%
http://118.69.50.154:80 健康值更新为： 316
成功: http://www.xiachufang.com/recipe/11478
11478 进度： 10.12%
http://65.52.158.214:80 健康值更新为： 371
成功: http://www.xiachufang.com/recipe/1075309
http://39.137.95.70:80 健康值更新为： 364
成功: http://www.xiachufang.com/recipe/102210286
1075309 http://65.52.158.214:80 健康值更新为： 372
成功: http://www.xiachufang.com/recipe/255215
进度： 10.15%
102210286 进度： 10.18%
255215 进度： 10.21%
http://163.172.152.52:8811 健康值更新为： 444
成功: http://www.xiachufang.com/recipe/100549109
100549109 进度： 10.24%
http://163.172.136.226:8811 健康值更新为： 260
成功: http://www.xiachufang.com/recipe/102118637
102118637 进度： 10.27%
http://84.17.47.204:80 健康值更新为： 358
成功: http://www.xiachufang.com/recipe/100408918
100408918 进度： 10.30%
http://39.137.95.71:80 健康值更新为： 442
成功: http://www.xiachufa

...

http://118.69.50.154:80 健康值更新为： 389
成功: http://www.xiachufang.com/recipe/100508029
100508029 进度： 87.55%
http://39.137.95.70:80 健康值更新为： 629http://39.137.95.70:80
成功: http://www.xiachufang.com/recipe/101829462
 健康值更新为： 630
成功: http://www.xiachufang.com/recipe/103168025
101829462 进度： 87.61%
103168025 进度： 87.61%
http://118.69.50.154:80 健康值更新为： 390
成功: http://www.xiachufang.com/recipe/268597
268597 进度： 87.64%
http://84.17.47.204:80 健康值更新为： 358
成功: http://www.xiachufang.com/recipe/101883964
101883964 进度： 87.67%
http://84.17.47.185:80 健康值更新为： 88
成功: http://www.xiachufang.com/recipe/101684270
101684270 进度： 87.70%
http://84.17.47.185:80 健康值更新为： 89
成功: http://www.xiachufang.com/recipe/22977
22977 进度： 87.73%
http://185.186.77.197:80 健康值更新为： 301
成功: http://www.xiachufang.com/recipe/100526621
100526621 进度： 87.76%
http://163.172.152.52:8811 健康值更新为： 394
成功: http://www.xiachufang.com/recipe/100103576
100103576 进度： 87.79%
http://163.172.136.226:8811 健康值更新为： 522
成功: http://www.xiachufang.com/recipe/1004

100577773 进度： 89.94%
http://118.69.50.154:80 健康值更新为： 396
成功: http://www.xiachufang.com/recipe/1000438
1000438 进度： 89.97%
http://84.17.47.185:80 健康值更新为： 96
成功: http://www.xiachufang.com/recipe/103699409
http://163.172.136.226:8811 健康值更新为： 535
成功: http://www.xiachufang.com/recipe/1097585
http://163.172.152.52:8811103699409 http://39.137.95.70:80 健康值更新为： 634
成功: http://www.xiachufang.com/recipe/104097219
健康值更新为： 进度： 90.00%
 402
成功: http://www.xiachufang.com/recipe/103706541
1097585 进度： 90.06%
104097219 进度： 90.06%
103706541 进度： 90.09%
http://118.69.50.154:80 健康值更新为： 397
成功: http://www.xiachufang.com/recipe/103779787
http://65.52.158.214:80 健康值更新为： 270
成功: http://www.xiachufang.com/recipe/1004013
103779787 进度： 90.12%
1004013 进度： 90.15%
http://39.137.95.71:80 健康值更新为： 695
成功: http://www.xiachufang.com/recipe/102814579
102814579 进度： 90.18%
http://51.158.165.18:8811 健康值更新为： 184
成功: http://www.xiachufang.com/recipe/246347
246347 进度： 90.22%
http://84.17.47.185:80 健康值更新为： 97
成功: http://www.xiachuf

263449 进度： 92.46%
http://65.52.158.214:80 健康值更新为： 280
成功: http://www.xiachufang.com/recipe/95967
95967 进度： 92.49%
http://118.69.50.154:80 健康值更新为： 405
成功: http://www.xiachufang.com/recipe/100458895
100458895 进度： 92.52%
http://84.17.47.204:80 健康值更新为： 372
成功: http://www.xiachufang.com/recipe/100567033
http://185.186.77.197:80 健康值更新为： 321
成功: http://www.xiachufang.com/recipe/100027260
100567033 进度： 92.55%
http://39.137.95.70:80 健康值更新为： 643
成功: http://www.xiachufang.com/recipe/1039549
100027260 进度： 92.58%
http://39.137.95.70:80 健康值更新为： 644
成功: http://www.xiachufang.com/recipe/100022680
1000226801039549 进度： 92.64%
 进度： 92.64%
http://84.17.47.185:80 健康值更新为： 106
成功: http://www.xiachufang.com/recipe/100428542
100428542 进度： 92.67%
http://84.17.47.185:80 健康值更新为： 107
成功: http://www.xiachufang.com/recipe/182865
182865 进度： 92.70%
http://39.137.95.71:80 健康值更新为： 702
成功: http://www.xiachufang.com/recipe/1014592
1014592 进度： 92.73%
http://51.158.165.18:8811 健康值更新为： 191
成功: http://www.xiachufang.com/recip

http://65.52.158.214:80 健康值更新为：103405451 进度： 95.06%
 292
成功: http://www.xiachufang.com/recipe/100602828
100602828 进度： 95.09%
http://185.186.77.197:80 健康值更新为： 334
成功: http://www.xiachufang.com/recipe/100345652
100345652 进度： 95.12%
http://163.172.136.226:8811 健康值更新为： 545
成功: http://www.xiachufang.com/recipe/103838014
103838014 进度： 95.15%
http://118.69.50.154:80 健康值更新为： 414
成功: http://www.xiachufang.com/recipe/100406553
100406553http://163.172.152.52:8811 健康值更新为： 414
成功: http://www.xiachufang.com/recipe/1088192
 进度： 95.18%
1088192 进度： 95.21%
http://39.137.95.71:80 健康值更新为： 712
成功: http://www.xiachufang.com/recipe/100571802
100571802 进度： 95.24%
http://118.69.50.154:80 健康值更新为： 415
成功: http://www.xiachufang.com/recipe/100105008
100105008 进度： 95.27%
http://39.137.95.71:80 健康值更新为： 713
成功: http://www.xiachufang.com/recipe/103138902
103138902 进度： 95.30%
http://185.186.77.197:80 健康值更新为： 335
成功: http://www.xiachufang.com/recipe/100553439
100553439 进度： 95.33%
http://118.69.50.154:80 健康值更新为： 416
成功: 

http://118.69.50.154:80 健康值更新为： 425
成功: http://www.xiachufang.com/recipe/100035144
http://39.137.95.70:80 健康值更新为： 659
成功: http://www.xiachufang.com/recipe/103654333
100035144 进度： 97.67%
http://65.52.158.214:80 健康值更新为： 299
成功: http://www.xiachufang.com/recipe/100481713
103654333 进度： 97.70%
100481713 进度： 97.73%
http://118.69.50.154:80 健康值更新为： 426
成功: http://www.xiachufang.com/recipe/1051240
1051240 进度： 97.76%
http://118.69.50.154:80 健康值更新为： 427
成功: http://www.xiachufang.com/recipe/1000038
1000038 进度： 97.79%
http://163.172.152.52:8811 健康值更新为： 422
成功: http://www.xiachufang.com/recipe/102201242
102201242 进度： 97.82%
http://163.172.152.52:8811 健康值更新为： 423
成功: http://www.xiachufang.com/recipe/102296279
102296279 进度： 97.85%
http://185.186.77.197:80 健康值更新为： 347
成功: http://www.xiachufang.com/recipe/100213345
100213345 进度： 97.88%
http://39.137.95.70:80 健康值更新为： 660
成功: http://www.xiachufang.com/recipe/100352761
100352761 进度： 97.91%
http://185.186.77.197:80 健康值更新为： 348
成功: http://www.xiachufang.com/

In [ ]:
# 70M，一个网页23k
rdb.hlen('html')  

3301

In [ ]:
title = soup.find('h1','page-title').text.strip()

'春饼 —薄如纸'

In [ ]:
author = soup.find('div','author')

In [ ]:
author.a['href']

'/cook/103536314/'

In [ ]:
author.span.text.strip()


'奇迹-75'

In [ ]:
soup.find('div','time').span.text.strip()

'2016-03-27 14:51:56'

In [ ]:
ingredient_list = [i.text.strip() for i in soup.find('div','ings').find_all('td','name')]
ingredient_list

['面粉（中筋或高筋）', '凉水或热水', '食用油']

In [ ]:
d = {'ings':ingredient_list}
d

{'ings': ['面粉（中筋或高筋）', '凉水或热水', '食用油']}

* redis无法存储多层级数据：{'ings': ['面粉（中筋或高筋）', '凉水或热水', '食用油']}
* 无法存储对象：soup

# TODO
- [ ] 专门爬懒饭的菜谱